In [1]:
from sqlalchemy import create_engine, inspect, ForeignKey, Column, Integer, Float, String, and_

from sqlalchemy.orm import relationship, remote, backref
from sqlalchemy.orm.collections import attribute_mapped_collection

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
class NetworkModel(Base):
    __tablename__ = 'models'

    id          = Column(Integer, primary_key = True, autoincrement=True)
    name        = Column(String, nullable=False)
    description = Column(String)
    
    wbs        = relationship('WBSRecord', backref='model', cascade = 'all, delete-orphan')
    actitivies = relationship('Activity' , backref='model', cascade = 'all, delete-orphan')
    links      = relationship('Link'     , backref='model', cascade = 'all, delete-orphan')

    #----------------------------------------------------------------------------------------------
    def describe(self):
        print(self.name, ':', self.description)
    
    #----------------------------------------------------------------------------------------------
    def copy(self, name=None):
        if not description:
            cp = NetworkModel(description=self.description, name='Copy of: ' + self.name)
        else:
            cp = NetworkModel(description=self.description, name=name)
        #cp.wbs        = [w.copy() for w in self.wbs]
        #cp.actitivies = [a.copy() for a in self.actitivies]
        #cp.links      = [l.copy() for l in self.links]
        raise NotImplementedError('FFFUUUU~~~')
        return cp
    
    #----------------------------------------------------------------------------------------------
    @property
    def _iwbs(self):
        return dict([(w.path, i) for i,w in enumerate(self.wbs)])
    
    def work(self, path):
        return self.wbs[self._iwbs[path]]
    
    #----------------------------------------------------------------------------------------------
    def delete_comment(self, path):
        w = self.wbs.pop(self._iwbs[path])
        del w
        inspect(self).session.flush()

###################################################################################################
class WBSRecord(Base):
    __tablename__ = 'wbs'

    id        = Column(Integer, primary_key=True, autoincrement=True)
    model_id  = Column(Integer, ForeignKey('models.id'))
    parent_id = Column(Integer, ForeignKey('wbs.id'))
    
    path      = Column(String, nullable=False, index=True)
    name      = Column(String, nullable=False)
    
    children = relationship("WBSRecord",
                            remote_side=parent_id,
                            back_populates='parent',
                            cascade='all'
                           )
    
    parent  = relationship("WBSRecord",
                            remote_side=id,
                            back_populates='children',
                           )
    
    activity = relationship('Activity', 
                            backref='wbs', 
                            cascade = 'all, delete-orphan', 
                            uselist=False)
    
    #----------------------------------------------------------------------------------------------
    def __init__(self, name, parent=None, model=None):
        #TODO: Сделать парвильный конструктор
        if parent:
            model = parent.model
            
        if not model:
            raise ValueError('Model id needed!!!')
        
        self.name   = name
        self.parent = parent
        self.model  = model
        self.path   = ''

        #That's shitty approach but I don't know how's better...
        inspect(self).session.flush()
        
        if parent:
            self.path += parent.path + '.'
        self.path += str(self.id)
       
    #----------------------------------------------------------------------------------------------
    def _propagate_path(self, new_path_base):
        if self.parent:
            self.path = self.parent.path + '.' + str(self.id)
        else:
            self.path = str(self.id)
            
        for c in self.children:
            c._propagate_path(self.path)

    #----------------------------------------------------------------------------------------------
    def move_to(self, new_path_base):
        if new_path_base != '':
            if new_path_base.startswith(self.path):
                raise ValueError('WTF R U doing motherfucker???')
            self.parent = self.model.comment(new_path_base)
        else:
            self.parent = None
        
        self._propagate_path(new_path_base)
        inspect(self).session.flush()
        
    #----------------------------------------------------------------------------------------------
    def copy(self):
        #TODO: Отладить это
        return WBSRecord(self.name, parent=self.parent, model=self.model)
    
    #----------------------------------------------------------------------------------------------
    def __repr__(self):
        return 'WBSRecord(model_id=%r id=%r, path=%r, name=%r)' % (
            self.model_id,
            self.id,
            self.path,
            self.name
        )

    #----------------------------------------------------------------------------------------------
    def dump(self, _level=0):
        return (
                '   ' * _level
                + repr(self)
                + "\n"
                + "".join([c.dump(_level + 1) for c in self.children])
        )

###################################################################################################
class Activity(Base):
    __tablename__ = 'activities'

    model_id = Column(Integer, ForeignKey('models.id'))
    id       = Column(Integer, ForeignKey('wbs.id'),    primary_key=True) #Как это копировать???
        
    in_links  = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.src_id)',
                             backref='src', cascade = 'all, delete-orphan')
    out_links = relationship('Link', 
                             primaryjoin='and_(Activity.model_id == Link.model_id, Activity.id == Link.dst_id)', 
                             backref='dst', cascade = 'all, delete-orphan')

    #Работы являются связями для событий
    src_id    = Column(Integer, ForeignKey('events.id'))
    dst_id    = Column(Integer, ForeignKey('events.id'))
    
    #CPM Data
    duration    = Column(Float, default=0.0) # Длительность работы
    early_start = Column(Float, default=0.0) # Ранний старт
    late_start  = Column(Float, default=0.0) # Поздний старт
    early_end   = Column(Float, default=0.0) # Ранний финиш
    late_end    = Column(Float, default=0.0) # Поздний финиш
    reserve     = Column(Float, default=0.0) # Резерв времени
    
    def __init__(self, wbs, src=None, dst=None):
        if not isinstance(wbs, WBSRecord):
            raise ValueError('WTF@!!')
            
        self.parent = parent
        self.src = src
        self.dst = dst

    #----------------------------------------------------------------------------------------------
    def copy(self, wbs):
        return Activity(wbs, src=self.src, dst=self.dst)

###################################################################################################
class Link(Base):
    __tablename__ = 'links'

    model_id  = Column(Integer, ForeignKey('models.id')    , primary_key=True)
    src_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    dst_id    = Column(Integer, ForeignKey('activities.id'),  primary_key=True)
    
    def __init__(src, dst):
        self.src = src
        self.dst = dst

    #----------------------------------------------------------------------------------------------
    def copy(self):
        return Link(src=self.src, dst=self.dst)

###################################################################################################
class Event(Base):
    __tablename__ = 'events'

    id       = Column(Integer,                          primary_key=True)
    model_id = Column(Integer, ForeignKey('models.id'), primary_key=True)

    in_activities  = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.src_id)',
                             backref='src')
    out_activities = relationship('Activity', 
                             primaryjoin='and_(Event.model_id == Activity.model_id, Event.id == Activity.dst_id)', 
                             backref='dst')
    
    early   = Column(Float, default=0.0)
    late    = Column(Float, default=0.0)
    reserve = Column(Float, default=0.0)

    #----------------------------------------------------------------------------------------------
    def copy(self):
        return Event(id=self.id)

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo = True)

In [4]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
Base.metadata.create_all(engine)

2021-08-29 21:40:58,605 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-29 21:40:58,606 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("models")
2021-08-29 21:40:58,606 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-29 21:40:58,607 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("models")
2021-08-29 21:40:58,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-29 21:40:58,609 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wbs")
2021-08-29 21:40:58,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-29 21:40:58,610 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("wbs")
2021-08-29 21:40:58,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-29 21:40:58,611 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("activities")
2021-08-29 21:40:58,612 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-29 21:40:58,613 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("activities")
2021-08-29 21:40:58,613 INFO sqlalchemy.engine.Engine [raw sql] ()
202

In [6]:
nm = NetworkModel(name='First model!')

In [ ]:
#План работы:
"""
1. Отлаить  работу WBSREcord, включая копирование
2. Отладить работу Activity
3. Отладить работу Event
4. Добавить узлы и ребра для визуализации сетевой модели.
"""